In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from datasets import load_dataset
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    prepare_model_for_kbit_training
)
import torch

# === Model & Tokenizer ===
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model = prepare_model_for_kbit_training(model)

# === Apply LoRA ===
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, peft_config)
model.enable_input_require_grads()

# === Load JSON dataset ===
dataset = load_dataset("json", data_files="scientific_summarization_chat.json")

# === Format dataset ===
def format_chat(example):
    prompt = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=True
    )
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    return {
        "input_ids": tokenized["input_ids"],
        "labels": tokenized["input_ids"]
    }

tokenized_dataset = dataset["train"].map(format_chat)

training_args = TrainingArguments(
    output_dir="./qlora-finetuned-tinyllama",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=200,  # ✅ Save every 200 steps
    report_to="none",
    save_total_limit=2  # Optional: keeps only the latest 2 checkpoints
)

# === Data collator ===
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# === Trainer ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# === Train ===
trainer.train()


c:\final\finetune\fineenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 2500/2500 [00:06<00:00, 397.49 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\final\finetune\fineenv\Lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, 

Step,Training Loss
100,2.192300
200,2.090600
300,2.057200
400,2.029600


c:\final\finetune\fineenv\Lib\site-packages\peft\utils\other.py:1110: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /TinyLlama/TinyLlama-1.1B-Chat-v1.0/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023217920590>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: b9e9fd77-0f0e-4b7d-94d7-aa88621481fb)') - silently ignoring the lookup for the file config.json in TinyLlama/TinyLlama-1.1B-Chat-v1.0.
  warnings.warn(
c:\final\finetune\fineenv\Lib\site-packages\peft\utils\save_and_load.py:236: UserWarning: Could not find a config file in TinyLlama/TinyLlama-1.1B-Chat-v1.0 - will assume that the vocabulary was not modified.
  warnings.warn(
c:\final\finetune\fineenv\Lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentra

TrainOutput(global_step=471, training_loss=2.0839363632688097, metrics={'train_runtime': 4395.4807, 'train_samples_per_second': 1.706, 'train_steps_per_second': 0.107, 'total_flos': 2.388706983936e+16, 'train_loss': 2.0839363632688097, 'epoch': 3.0})

In [2]:
trainer.model.save_pretrained("./qlora-finetuned-tinyllama-final")
tokenizer.save_pretrained("./qlora-finetuned-tinyllama-final")

('./qlora-finetuned-tinyllama-final\\tokenizer_config.json',
 './qlora-finetuned-tinyllama-final\\special_tokens_map.json',
 './qlora-finetuned-tinyllama-final\\chat_template.jinja',
 './qlora-finetuned-tinyllama-final\\tokenizer.json')